In [1]:
# creating paths to src and data folders in the repo
import sys
import pathlib
src_path = pathlib.Path().absolute().parent.parent / "src"
data_path = pathlib.Path().absolute().parent.parent / "data"

# train test split paths
train_path = data_path / 'tts/train'
test_path = data_path / 'tts/test'
val_path = data_path / 'tts/val'

# add src path to sys.path so it is searched in import statements
sys.path.append(str(src_path))

# basic imports for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# silence max image size warning
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 

# import modeling packages
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# modeling metrics
from sklearn.metrics import classification_report, confusion_matrix

changes to data generators:
    
    train batch_size 100 -> 20
    
    test batch_size 33 -> 50
    
    added val_generator

In [2]:
train_generator = ImageDataGenerator().flow_from_directory(str(train_path),
                                                           target_size=(150, 150),
                                                           batch_size=20,
                                                           class_mode='categorical')
test_generator = ImageDataGenerator().flow_from_directory(str(test_path),
                                                          target_size=(150, 150),
                                                          batch_size=50,
                                                          class_mode='categorical',
                                                          shuffle=False)
val_generator = ImageDataGenerator().flow_from_directory(str(val_path),
                                                          target_size=(150, 150),
                                                          batch_size=109,
                                                          class_mode='categorical',
                                                          shuffle=False)

Found 4000 images belonging to 5 classes.
Found 500 images belonging to 5 classes.
Found 545 images belonging to 5 classes.


changes to model architecture:
    
    num conv2d layers 2
    
    1st dense layer dropout .25 -> 0
    
    num output units 6 -> 5

matching the new number of classes

In [3]:
model = models.Sequential()
model.add(layers.Conv2D(30, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(30, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(40, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 30)      840       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 30)        8130      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 30)        0         
_________________________________________________________________
flatten (Flatten)            (None, 38880)             0         
_________________________________________________________________
dense (Dense)                (None, 40)                1555240   
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2

changes to fit:
    
    steps_per_epoch 57 -> 200
    
    validation_steps 11 -> 10
    
    epochs 4 -> 5

In [5]:
model.fit(
        train_generator,
        steps_per_epoch=200,
        epochs=4,
        validation_data=test_generator,
        validation_steps=10)

Epoch 1/4
 16/200 [=>............................] - ETA: 7:14 - loss: 103.0494 - accuracy: 0.1562

/opt/anaconda3/envs/capstone-env/lib/python3.6/site-packages/PIL/Image.py:952: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


200/200 [==============================] - 611s 3s/step - loss: 9.7926 - accuracy: 0.2637 - val_loss: 1.5872 - val_accuracy: 0.2820
Epoch 2/4
200/200 [==============================] - 588s 3s/step - loss: 1.4340 - accuracy: 0.4160 - val_loss: 1.5812 - val_accuracy: 0.3160
Epoch 3/4
200/200 [==============================] - 610s 3s/step - loss: 1.1046 - accuracy: 0.6000 - val_loss: 1.7778 - val_accuracy: 0.3280
Epoch 4/4
200/200 [==============================] - 606s 3s/step - loss: 0.6363 - accuracy: 0.7903 - val_loss: 2.1691 - val_accuracy: 0.3600


In [6]:
confusion_matrix(test_generator.classes, np.argmax(model.predict(test_generator), axis=1))

array([[44, 11,  9, 17, 19],
       [32, 34, 11, 12, 11],
       [17, 11, 38, 14, 20],
       [30,  6, 11, 25, 28],
       [24, 11, 12, 14, 39]])